<p align="center">
<img src="img/lasair.png" alt="drawing" width="50"/>
</p>
<h1 align="center">  Lasair API Recipes </h1>


---

## What you'll learn

This notebook will teach you the essentials for querying the Lasair API and finding the data you want:

* How to do a Cone Search
* How to find a specific `diaObjectId`
* What data you get back and how to navigate it
* How to make more complex queries, including Sherlock and Annotators

## Pre-requesites

### **Have you got a Lasair Token?**

**_Why do I need a token to stream public alerts?_** Because without authentication systems, bad actors can mess with our servers. But creating a Lasair profile is **Free and Public** for everyone. You can do this on the [Lasair Platform](http://lasair.lsst.ac.uk/) (go to Sing-In page).

Then you will be able to **see and copy your token** from your [profile](http://lasair.lsst.ac.uk/profile) page. 

**You should not share your token or save it in public repositories e.g. GitHub**.  One of the easies things to do so you don't have to keep pasting it, is to save it on your local machine in your shell environment. For example add this to your `bashrc` or `bash_profile`

```bash
export LASAIR_LSST_TOKEN=blablablablablYOUROTKENHEREblablabla
```

### **Have you installed the `lasair` client ?**

You can do this through pip!

```bash
pip install lasair
```

---


<h2 align="center">  1. Cone Search </h1>




In [96]:
from lasair import lasair_client as lapi
import os

If you have set your environment variable you can get it into python very easily!   

In [97]:
API_TOKEN = os.getenv('LASAIR_LSST_TOKEN')

if API_TOKEN is None:
    print("No Token found. Check Spelling. Note that if you have just added your token to your environment variables, you may need to restart your terminal so your shell settings are reloaded. ")

Now we create our lasair API client instance, it takes in our token and the location of the Lasair API on the internet (endpoint).

In [98]:
L = lapi(API_TOKEN, 
         endpoint="https://lasair-lsst-dev.lsst.ac.uk/api/",
         #PROD endpoint='https://api.lasair.lsst.ac.uk/api'
         )

Let's pick a random sky location, and set our search radius to 15 arcseconds

In [99]:
ra = 149.164175
dec= 2.649324
radius = 15

There are **3 types of Cone Searches**:
- Count: Number of alerts found within the radius
- Nearest: Which object is nearest (but still within the radius! If radius is too small this can be 0)
- All: Every alert that fulfill the criteria


### Count


In [100]:
result = L.cone(ra, 
                dec, 
                radius, 
                requestType='count'
                )

print('Found %d at radius %f' % (result['count'], radius))

Found 4 at radius 15.000000


### Nearest

In [102]:
result = L.cone(ra, dec, radius, requestType='nearest')
if 'nearest' in result:
    obj = result['nearest']
    print('Found %s at separation %.2f with radius %f' % (obj['object'], obj['separation'], radius))
else:
    print('No object found at radius %f' % radius)

Found 313998569323888791 at separation 0.00 with radius 15.000000


### ALL

In [103]:
print('At radius %f' % radius)
result = L.cone(ra, dec, radius, requestType='all')
if 'object' in result['nearest']:
    print('nearest ', result['nearest']['object'], \
        'at separation %.2f' % result['nearest']['separation'])
    objects = result['objects']
    for obj in objects:
        print(obj['object'], "%.2f" % obj['separation'])
else:
    print('no object found')

At radius 15.000000
nearest  313998569323888791 at separation 0.00
313998569323888791 0.00
169883381784903846 0.29
314011776099811451 1.63
314003015436075376 3.45



<h2 align="center">  2. Query by diaObjectId and Getting Familiar with the data</h1>

In [104]:
result = L.object(313963406691402058, lasair_added=True)

## What data do we get?

If we just print the results we can see that we have a big bundle of JSON, let's walk through the key elements

In [105]:
result


{'diaObjectId': '313963406691402058',
 'lasairData': {'nDiaSources': 187,
  'firstDiaSourceMjdTai': 61049.32267149135,
  'lastDiaSourceMjdTai': 61091.22429989655,
  'glat': 41.6451,
  'ebv': 0.0216602,
  'rasex': '09:55:20.980',
  'decsex': '03:12:45.992',
  'ec_lon': 149.84676879851827,
  'ec_lat': -8.864258592844049,
  'g_lon': 234.73006275974515,
  'g_lat': 41.6451382157101,
  'now_mjd': '61091.61',
  'mjdmin_ago': 42.28351946901239,
  'mjdmax_ago': 0.3818910638074158,
  'discMjd': 61049.32267149135,
  'discUtc': '2026-01-09 07:44:38',
  'discMag': '2176.10±337.12',
  'discFilter': 'i',
  'latestMjd': 61091.28504779168,
  'latestUtc': '2026-02-20 06:50:28',
  'latestMag': '5461.46±542.54',
  'latestFilter': 'i',
  'peakMjd': 61090.199582002075,
  'peakUtc': '2026-02-19 04:47:23',
  'peakMag': '6526.54±463.58',
  'peakFilter': 'z',
  'sherlock': {'diaObjectId': 313963406691402058,
   'classification': 'ORPHAN',
   'association_type': None,
   'catalogue_table_name': None,
   'catalog

Let's see the top lop keys

In [106]:
result.keys()

dict_keys(['diaObjectId', 'lasairData', 'diaObject', 'diaSourcesList', 'diaForcedSourcesList'])

`diaObjectId` is a simple field, just what we request, also it's the primary index used by the Lasair database.

In [108]:
result['diaObjectId']

'313963406691402058'

### ASK ROY AND LASAIR DEVS WHAT AM I LOOKING AT HERE - why different from everything else?

In [112]:
result['diaObject']

{'ra': 148.83741827252254,
 'decl': 3.212775635026692,
 'firstDiaSourceMjdTai': 61049.32267149135,
 'lastDiaSourceMjdTai': 61091.22429989655}

In [113]:
result['diaForcedSourcesList']

[{'midpointMjdTai': 61088.211979366155,
  'band': 'r',
  'psfFlux': 4016.017333984375,
  'psfFluxErr': 158.10763549804688},
 {'midpointMjdTai': 61088.212443683144,
  'band': 'r',
  'psfFlux': 3924.970703125,
  'psfFluxErr': 149.4063720703125},
 {'midpointMjdTai': 61088.21739376974,
  'band': 'z',
  'psfFlux': 5133.67333984375,
  'psfFluxErr': 381.5521545410156},
 {'midpointMjdTai': 61090.1077735384,
  'band': 'i',
  'psfFlux': 5258.7353515625,
  'psfFluxErr': 309.8955993652344},
 {'midpointMjdTai': 61090.180597784965,
  'band': 'i',
  'psfFlux': 5386.8271484375,
  'psfFluxErr': 269.358154296875},
 {'midpointMjdTai': 61090.18153057039,
  'band': 'i',
  'psfFlux': 4912.7607421875,
  'psfFluxErr': 265.480712890625},
 {'midpointMjdTai': 61090.181996839354,
  'band': 'i',
  'psfFlux': 4960.11376953125,
  'psfFluxErr': 242.54441833496094},
 {'midpointMjdTai': 61090.182463713754,
  'band': 'i',
  'psfFlux': 5441.931640625,
  'psfFluxErr': 257.9368896484375},
 {'midpointMjdTai': 61090.18292844

In [111]:
result['diaSourcesList']

[{'diaSourceId': 170032928435732713,
  'midpointMjdTai': 61091.28504779168,
  'band': 'i',
  'psfFlux': 5461.46484375,
  'psfFluxErr': 542.5358276367188,
  'reliability': 0.9929967522621155},
 {'diaSourceId': 170032915773129911,
  'midpointMjdTai': 61091.228487750166,
  'band': 'r',
  'psfFlux': 3740.444580078125,
  'psfFluxErr': 206.02182006835938,
  'reliability': 0.9844580292701721},
 {'diaSourceId': 170032915635241270,
  'midpointMjdTai': 61091.22802527272,
  'band': 'r',
  'psfFlux': 4073.19970703125,
  'psfFluxErr': 313.6912536621094,
  'reliability': 0.9766911864280701},
 {'diaSourceId': 170032915363135541,
  'midpointMjdTai': 61091.227095847906,
  'band': 'r',
  'psfFlux': 3850.011962890625,
  'psfFluxErr': 288.5678405761719,
  'reliability': 0.9887726902961731},
 {'diaSourceId': 170032915095748678,
  'midpointMjdTai': 61091.22616138874,
  'band': 'r',
  'psfFlux': 3878.681396484375,
  'psfFluxErr': 260.0511779785156,
  'reliability': 0.9842692613601685},
 {'diaSourceId': 17003

In [110]:
result['lasairData'].keys()

dict_keys(['nDiaSources', 'firstDiaSourceMjdTai', 'lastDiaSourceMjdTai', 'glat', 'ebv', 'rasex', 'decsex', 'ec_lon', 'ec_lat', 'g_lon', 'g_lat', 'now_mjd', 'mjdmin_ago', 'mjdmax_ago', 'discMjd', 'discUtc', 'discMag', 'discFilter', 'latestMjd', 'latestUtc', 'latestMag', 'latestFilter', 'peakMjd', 'peakUtc', 'peakMag', 'peakFilter', 'sherlock', 'TNS', 'annotations', 'imageUrls'])

<h2 align="center">  3. Advanced Queries </h1>

You can build advanced SQL-like queries for more refinded searches. In this example we will look at finding objects based on their [Sherlock](https://lasair.readthedocs.io/en/main/core_functions/sherlock.html#sherlock--sky-context-) Classification, which is based on catalogue cross-matching. 


### Select your data
First let's define the **columns we want to get out of our query**: here it will be the `diaObjectId`, Ra and Dec.

Since we're doing an SQL query, we need to know the correct name for the table and columns, which you can find on the [Lasair Website Schema Browser](https://lasair-lsst-dev.lsst.ac.uk/schema/) 


![schema_browser](img/schema_browser.png)

In [89]:
selected = 'objects.diaObjectId, objects.ra, objects.decl'

### Select your conditions

The next thing we want to do is format our conditions, where we want the data **only if Sherlock said it could be a SN** (i.e. near a galaxy but not at the nucleus).

Scrollking down on our Schema Broswer we can see there is a `sherlock_classification` with a `classification` column. We'll need that == `SN` 

![schema_browser](img/schema_browser_sherlock.png)


In [90]:
conditions = 'classification="SN"'

*[FAQ]: Why is it not `sherlock_classification.classification="SN"`?*

### List the tables needed for your query

In [91]:
tables = 'objects,sherlock_classifications'

### Call your query!

You'll notice here we also set a limit of 8, because a lot of objects actually match this condition.


In [92]:
results = L.query(selected, tables, conditions, limit = 8)

In [93]:
results

[{'diaObjectId': 169298433161560316,
  'ra': 10.145744291286427,
  'decl': -44.754963238154986},
 {'diaObjectId': 169298433201930256,
  'ra': 10.250262019119214,
  'decl': -43.299100965937846},
 {'diaObjectId': 169298436674813978,
  'ra': 8.796281610710041,
  'decl': -43.98532171023203},
 {'diaObjectId': 169298436985192488,
  'ra': 8.313652586179952,
  'decl': -43.036609073180145},
 {'diaObjectId': 169298437172363418,
  'ra': 9.666946696565965,
  'decl': -45.64004534870912},
 {'diaObjectId': 169298437307105370,
  'ra': 10.041701764645088,
  'decl': -45.53332938361457},
 {'diaObjectId': 169298437505810433,
  'ra': 7.8915515318678215,
  'decl': -44.500186689763105},
 {'diaObjectId': 169298437514723368,
  'ra': 8.631704335763443,
  'decl': -42.617216230724495}]

<h2 align="center">  4. Querying an Annotator </h1>

[TBD]

---

<h2 align="center"> What's Next </h2>


...

<h2 align="center"> How to Get Help </h2>

If you have a question or need any help, you can find past questions or as a new one on the **[Community Forum](https://community.lsst.org/c/support/support-lasair/55)**.


![communityforum](img/community_forum.png)
